In [1]:
import awscli
import boto3

import numpy as np
import pandas as pd
import cv2
from PIL import Image
from io import BytesIO
import io
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import models, transforms
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

from tqdm.notebook import tqdm

In [2]:
BUCKET_NAME = 'msds-fashion' 

# enter authentication credentials
s3 = boto3.client('s3', aws_access_key_id = '####', 
                          aws_secret_access_key= '####')

In [3]:
response = s3.get_object(Bucket= BUCKET_NAME, Key = 'data/train_labels.csv')
train = pd.read_csv(response.get('Body'))
response = s3.get_object(Bucket= BUCKET_NAME, Key = 'data/test_labels.csv')
test = pd.read_csv(response.get('Body'))

In [4]:
len(train),len(test)

(249222, 40000)

In [5]:
len(np.unique(train.category_name))

46

In [6]:
device_no = 0
if torch.cuda.is_available() == True:
    device = torch.device(device_no)
else:
    device = torch.device('cpu')
device

device(type='cpu')

In [7]:
class FashionDataset(Dataset):
  def __init__(self,df):
    self.df = df
    self.s3 = boto3.resource('s3', aws_access_key_id = '####', 
                          aws_secret_access_key= '####',
                          region_name= 'us-west-2')
    
    self.bucket = self.s3.Bucket('msds-fashion')
    self.label_dict = {o:i for i,o in enumerate(np.unique(train.category_name))}

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    key = '/'.join(row.image_name.split('\\'))
    obj = self.bucket.Object(key).get()['Body'].read()
    image_arr = cv2.imdecode(np.asarray(bytearray(obj)), cv2.IMREAD_COLOR)
    
    img = cv2.cvtColor(image_arr, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = np.transpose(img, (2,0,1))

    img = torch.tensor(img/255.).float()
    label = torch.tensor(self.label_dict[row['category_name']])

    return img, label

In [8]:
train_ds = FashionDataset(train)
train_dl = DataLoader(train_ds, batch_size= 1000, shuffle=True)
valid_ds = FashionDataset(test)
valid_dl = DataLoader(valid_ds, batch_size= 1000, shuffle=False)

In [9]:
def one_pass(model, dataloader, optimizer, lossFun, device, backwards=True, print_loss=False):
    
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0
    for x, y in tqdm(dataloader):
        
        # send labelled data to the device
        x, y = x.to(device), y.to(device)
        
        y_pred = model(x)
        loss = lossFun(y_pred, y)
        total_loss += loss.item()
        
        if backwards == True:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    avg_loss = total_loss / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
    
    return avg_loss

def one_pass_acc(model, dataloader, num_points):
  model.eval()
  total_incorrect = 0

  softmax = nn.LogSoftmax(dim=1)

  for x, y in dataloader:
    x,y = x.to(device), y.to(device)
    y_pred = softmax(model(x))
    y_pred = torch.argmax(y_pred, dim=1)

    total_incorrect += torch.count_nonzero(y-y_pred).item()

  percent_wrong = total_incorrect / num_points
  return 1- percent_wrong

## Load fine-tuned Parameters for ResNet-18

In [10]:
model = models.resnet18(pretrained=True).to(device)
model.fc = nn.Linear(512,46)
print(model.fc)
pretrained = torch.load('models/resnet18_full_1epoch.pth', map_location='cpu')
model.load_state_dict(pretrained)

Linear(in_features=512, out_features=46, bias=True)


<All keys matched successfully>

In [11]:
# getting accuracy missed due to typo
train_acc = one_pass_acc(model.to(device), train_dl, len(train_ds))
valid_acc = one_pass_acc(model.to(device), valid_dl, len(valid_ds))
print('Train Acc: ', train_acc)
print('Valid Acc: ', valid_acc)

Train Acc:  0.5044578729004663
Valid Acc:  0.496275
